In [1]:
import sys
import soundcloud
import networkx as nx

In [2]:
from functools import partial
from utils import get_results, handle_http_errors

In [3]:
def getAttr(resource, attr):
    if hasattr(resource, attr): return getattr(resource, attr)
    return None

getUsername = partial(getAttr, attr='username')
getid = partial(getAttr, attr='id')
getUserid = partial(getAttr, attr='user_id')

In [4]:
def profile2id(profile):
    print("get user id result is {}".format(getUserid(profile)))
    print("get id result is {}".format(getid(profile)))
    return getUserid(profile) if isinstance(getUserid(profile), int) else getid(profile)

In [5]:
def profile2username(profile):
    return getUsername(profile) if isinstance(getUsername(profile), str) else str(getid(profile))

In [6]:
# A global profile graph used to iterate through the various algorithms.
# Each node is profile id, with edges weighted by activity between then.
profile_graph = nx.MultiDiGraph()


In [50]:
client = soundcloud.Client(client_id='454aeaee30d3533d6d8f448556b50f23')
name = input("Enter a soundcloud artist to analyze: ")

Enter a soundcloud artist to analyze: Sybyr


In [7]:
# Artist of interest

search = client.get('/users/', q = name)[0]

In [8]:
search.__dict__

{'obj': {'avatar_url': 'https://i1.sndcdn.com/avatars-cIS3ELGYY7j2e0q9-OEySHA-large.jpg',
  'id': 13377746,
  'kind': 'user',
  'permalink_url': 'https://soundcloud.com/sybyr',
  'uri': 'https://api.soundcloud.com/users/13377746',
  'username': 'Sybyr',
  'permalink': 'sybyr',
  'last_modified': '2020/08/16 23:02:35 +0000',
  'first_name': 'Sybyrmusic.com',
  'last_name': '',
  'full_name': 'Sybyrmusic.com',
  'city': '',
  'description': 'bookings/email: sybyrcentral@gmail.com\nAnti-World: http://antiworld.co (@antiworldglobe) on all networks\n(Prod. by Landfill) at http://prodbylandfill.com',
  'country': 'United States',
  'track_count': 437,
  'public_favorites_count': 0,
  'followers_count': 52213,
  'followings_count': 23,
  'plan': 'Pro Unlimited',
  'myspace_name': None,
  'discogs_name': None,
  'website_title': 'click Here',
  'website': 'http://antiworld.lnk.to/SYBYR',
  'reposts_count': 105,
  'comments_count': 110,
  'online': False,
  'likes_count': 0,
  'playlist_count':

In [9]:
print("Artist interpreted as: %s" % getUsername(search))

Artist interpreted as: Sybyr


In [10]:
uid2username_cache = {}
tid2username_cache = {}
cid2username_cache = {}
tid2userid_cache = {}
cid2userid_cache = {}

In [11]:
@handle_http_errors
def userid2username(uid):
    global uid2username_dict
    username = uid2username_cache.get(uid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    uid_str = str(uid)
    get = '/users/{}'.format(uid_str)
    result = client.get(get)
    username = result.username
    uid2username_cache[uid] = username
    return username

@handle_http_errors
def trackid2username(tid):
    global tid2username_dict
    username = tid2username_cache.get(uid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    tid_str = str(tid)
    get = '/tracks/{}'.format(tid_str)
    result = client.get(get)
    username = result.user['username']
    tid2username_cache[tid] = username
    return username

@handle_http_errors
def commentid2username(cid):
    global cid2username_dict
    username = cid2username_cache.get(uid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    cid_str = str(cid)
    get = '/comments/{}'.format(cid_str)
    result = client.get(get)
    username = result.user['username']
    cid2username_cache[cid] = username
    return username
    
@handle_http_errors    
def trackid2userid(tid):
    global tid2userid_dict
    userid = tid2userid_cache.get(tid, None)
    if userid is not None: 
        return userid
    # username is none, we don't have it in cache
    
    tid_str = str(tid)
    get = '/tracks/{}'.format(tid_str)
    result = client.get(get)
    userid = result.user['id']
    tid2userid_cache[tid] = userid
    return userid

def commentid2userid(cid):
    global cid2userid_dict
    username = cid2username_cache.get(cid, None)
    if username is not None: 
        return username
    # username is none, we don't have it in cache
    
    cid_str = str(cid)
    get = '/comments/{}'.format(cid_str)
    result = client.get(get)
    userid = result.user['id']
    cid2userid_cache[cid] = userid
    return userid
    

# get social data functions

In [12]:
def getRelationships(profile, client, url): 
    return get_results(client, url)

@handle_http_errors
def getFollows(profile):
    # get all profiles a user follows
    follows = get_results(client, '/users/{0:s}/followings/'.format(str(profile)))
    return follows

@handle_http_errors
def getFollowers(profile):
    # get all profiles that follow a user
    followers = get_results(client, '/users/{0:s}/followers/'.format(str(profile)))
    return followers

@handle_http_errors
def getUserFavorites(profile):
    user_favorites = get_results(client, '/users/{0:s}/favorites/'.format(str(profile)))
    return user_favorites

@handle_http_errors
def getUserComments(profile):
    comments = get_results(client, '/users/{0:s}/comments/'.format(str(profile)))
    return comments

@handle_http_errors
def getTracks(profile):
    tracks = get_results(client, '/users/{0:s}/tracks/'.format(str(profile)))
    return tracks

@handle_http_errors
def getTrackComments(track):
    track_comments = get_results(client, '/tracks/{0:s}/comments/'.format(str(track)))
    return track_comments

@handle_http_errors
def getTrackFavoriters(track):
    track_favoriters = get_results(client, '/tracks/{0:s}/favoriters/'.format(str(track)))
    return track_favoriters

# metadata functions

In [13]:
def getUserInfo(profile):
    info = ['id','permalink', 'username', 'avatar_url', 'country', 'city', 
            'website', 'track_count', 'followers_count', 'followings_count']
    return {i: getAttr(profile, i) for i in info}

def getTrackInfo(track):
    info = ['id', 'user_id', 'created_at', 'streamabale', 'downloadable',
            'playback_count', 'download_count', 'favoritings_count','comment_count']
    return {i: getAttr(track, i) for i in info}

def getUserFavInfo(fav):
    info = ['id', 'user_id','last_modified']
    return {i: getAttr(fav, i) for i in info}

def getTrackFavInfo(fav):
    info = ['id', 'username', 'last_modified']
    return {i: getAttr(fav, i) for i in info}

def getUserCommInfo(comm):
    info = ['id', 'user_id', 'track_id', 'timestamp']
    return {i: getAttr(comm, i) for i in info}

def getTrackCommInfo(comm):
    info = ['id', 'user_id', 'track_id','timestamp']
    return {i: getAttr(comm, i) for i in info}

# build graph functions: Networkx

In [14]:
# A global profile graph used to iterate through the various algorithms.
# Each node is profile id, with edges weighted by activity between then.
profile_graph = nx.MultiDiGraph()

In [15]:
def addProfileNx(profile, profileGraph):
    profileGraph.add_node(profile)
    return profileGraph

def addPairNx(profile, neighbor, profileGraph):
    addProfileNx(profile)
    addProfileNx(neighbor)

def addFollowsNx(profile, follows, profileGraph):
    for follow in follows:
        # enforce that already existing follows don't get added again
        if not profileGraph.has_edge(profile, follow, 'soundcloud_follow'):
            profileGraph.add_edge(profile, follow, 'soundcloud_follow',weight=1)

# TODO: enforce that the same favorite isn't counted twice
# easy solve: keep a list of tracks as an edge attribute
def addUserFavoritesNx(profile, user_favorites, profileGraph):
    for user_favorite in user_favorites:
        if profileGraph.has_edge(profile, user_favorite, 'soundcloud_favorite'):
            profileGraph.edges[profile, user_favorite,'soundcloud_favorite']['weight']+=1
        else:
            profileGraph.add_edge(profile, user_favorite, 'soundcloud_favorite',weight=1)

# TODO: enforce that the same comment isn't counted twice
# easy solve: keep a dictionary of track-comments pairs as an edge attribute
def addUserCommentsNx(profile, user_comments, profileGraph):
    for user_comment in user_comments:
        if profileGraph.has_edge(profile, user_comment, 'soundcloud_comment'):
            profileGraph.edges[profile, user_comment,'soundcloud_comment']['weight']+=1
        else:
            profileGraph.add_edge(profile, user_comment, 'soundcloud_comment',weight=1)

def addFollowersNx(profile, followers, profileGraph):
    for follower in followers:
        if not profileGraph.has_edge(follower, profile, 'soundcloud_follow'):
            profileGraph.add_edge(follow, profile, 'soundcloud_follow',weight=1)
            
def addTrackCommenters(profile, track_commenters, profileGraph):
    for track_commenter in track_commenters:
        if profileGraph.has_edge(track_commenter, profile, 'soundcloud_comment'):
            profileGraph.edges[track_commenter, profile,'soundcloud_comment']['weight']+=1
        else:
            profileGraph.add_edge(track_commenter, profile, 'soundcloud_comment',weight=1)
            

In [16]:
def printGraph(G):
    for profile, neighbor, key, weight in G.edges(data=True,keys=True):
        print("Profile: {}".format(userid2username(profile)))
        print("Neighbor: {}".format(userid2username(neighbor)))
        print(key)
        print(weight)
#     for profile in G.nodes():
#         if profile:
#             try:
#                 profile_name = id2username(profile)
#                 print("\t", "Printing: %s (%s)" % (profile_name, profile))
#                 # why does successors return an iterator instead of a list? getting fancy for no reason
#                 follows = [follow for follow in G.successors(profile)]
#                 followers = [follower for follower in G.predecessors(profile)]
#                 try:    
#                     print("\t", profile_name + " has " + str(len(follows)) + " followings")
#                     print("\t", profile_name + " follows " + ", ".join(map(lambda x: id2username(x), follows)))
#                 except TypeError: 
#                     print("No follows here!")
#                 try:    
#                     print("\t", profile_name + " has " + str(len(followers)) + " followers")
#                     print("\t", profile_name + " is followed by " + ", ".join(map(lambda x: id2username(x), followers)))
#                 except TypeError:
#                     print("No followers here!")
#                     print("-"*40)
#             except UnicodeError:
#                 print("Artist's username not found" )   

### 1b. Testing Functions

In [17]:
# list of profiles to query
profiles_to_query = [search.id]
profiles_to_query

[13377746]

In [18]:
depth = 1
i = 0

# list of profiles we could not query
unavailable_profiles = []

Only uncomment the code in the loop if you are willing to wait a while to run the cell below:

In [19]:
for t in range(depth):
    print("Iteration " + str(t))
    profiles_to_query = list(set(profiles_to_query))
    
    for profile in profiles_to_query:
        username = userid2username(profile)
        if username:
            print("\t", "Enqueueing: %s (%s)" % (username, profile))
            addProfileNx(profile, profile_graph)

#             encountered_follows = getFollows(profile)
#             encountered_follow_profiles = [follow.id for follow in encountered_follows]
#             print("encountered follows: " + ", ".join([userid2username(profile) for profile in encountered_follow_profiles]))
#             print('\n')
#             addFollowsNx(profile, encountered_follow_profiles, profile_graph)

#             encountered_user_favorites = getUserFavorites(profile)
#             encountered_user_favorite_profiles = [getUserid(track) if isinstance(getUserid(track), int) else track.user_id for track in encountered_user_favorites]
#             print("encountered user favorite profiles: " + ", ".join([userid2username(profile) for profile in encountered_user_favorite_profiles]))
#             print('\n')
#             addUserFavoritesNx(profile, encountered_user_favorite_profiles, profile_graph)

#             encountered_user_comments = getUserComments(profile)
#             encountered_user_comment_profiles = [trackid2userid(comment.track_id) for comment in encountered_user_comments]
#             print("encountered user comment profiles: " + ", ".join([userid2username(profile) for profile in encountered_user_comment_profiles]))
#             addUserCommentsNx(profile, encountered_user_comment_profiles, profile_graph)
#             print('\n')

#             encountered_followers = getFollowers(profile)
#             encountered_follower_profiles = [follower.id for follower in encountered_followers]
#             print("encountered followers: " + ", ".join([userid2username(profile) for profile in encountered_follower_profiles]))
#             print('\n')
#             addFollowersNx(profile, encountered_follower_profiles, profile_graph)

            encountered_tracks = getTracks(profile)

            encountered_track_comments = []
            encountered_track_favoriters = []

            for track in encountered_tracks:
                print("getting comments / favoriters from user track, {}".format(track.title))
                encountered_track_comments += getTrackComments(track.id)
                encountered_track_favoriters += getTrackFavoriters(track.id)

            print(encountered_track_comments[0].__dict__)
            encountered_track_commenters = [(comment.user['id'], comment.user['username']) for comment in encountered_track_comments]
            encountered_track_commenter_ids = [encountered_track_commenter[0] for encountered_track_commenter in encountered_track_commenters]
            encountered_track_commenter_usernames = [encountered_track_commenter[1] for encountered_track_commenter in encountered_track_commenters]
            print("encountered track comments: " + ", ".join(encountered_track_commenter_usernames))
            print('\n')
            addTrackCommenters(profile, encountered_track_commenter_ids, profile_graph)
#             print(encountered_track_favoriters[0].__dict__)

#             print("encountered track favoriters: " + ", ".join([getUsername(favoriter) if isinstance(getUsername(favoriter), str) else favoriter.user['username'] for favoriter in encountered_track_favoriters]))
        else:
            print("\t", "Artist ID %s is not query-able" % profile)
            unavailable_profiles.append(profile)            

Iteration 0
	 Enqueueing: Sybyr (13377746)
getting comments / favoriters from user track, How I Feel About Now (Play For The Band)
getting comments / favoriters from user track, ProdbyLandfill.com (feat. Landfill)
getting comments / favoriters from user track, Do It Again. Prod Kiddbeatz
getting comments / favoriters from user track, Back Prod Babe
getting comments / favoriters from user track, Let The Church Say Bang ProdbyLandfill.com
getting comments / favoriters from user track, Level - Headed Prod Kiddbeatz
getting comments / favoriters from user track, Unknowns ProdbyLandfill.com
getting comments / favoriters from user track, Black & Mild Spirits ProdbyLandfill.com
getting comments / favoriters from user track, Sleepy Cherry Prod Babe
getting comments / favoriters from user track, Land Of The Land ProdbyLandfill.com
getting comments / favoriters from user track, Bipolar Unstable Soldier Prod Kiddbeatz
getting comments / favoriters from user track, Penny Tail 2 ProdbyLandfill.com


getting comments / favoriters from user track, Unh Unh (Prod. Landfill)
getting comments / favoriters from user track, Slowpokes Do Not Eat (Prod. Landfill)
getting comments / favoriters from user track, I Don't Know, Feel Diluted (Prod. Landfill)
getting comments / favoriters from user track, Mad Max Mode (Prod. Y$57)
getting comments / favoriters from user track, Weeping (Prod. Landfill)
getting comments / favoriters from user track, Boost (Prod. Landfill)
getting comments / favoriters from user track, Buckle Down (Prod. Landfill)
getting comments / favoriters from user track, Draino (Prod. Shark)
getting comments / favoriters from user track, Still (Prod. Landfill)
getting comments / favoriters from user track, What A Why Time To Die (Prod. Landfill)
getting comments / favoriters from user track, Save Ya Knuckle Talk (Prod. Landfill)
getting comments / favoriters from user track, Rat Tattoo (Prod. Cxrpse)
getting comments / favoriters from user track, OH Freestyle (Prod. Landfill)
g

In [20]:
printGraph(profile_graph)

Profile: Sybyr
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: wlfgrl
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: quanison
Neighbor: Sybyr
soundcloud_comment
{'weight': 4}
Profile: vnstable
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: LyFE$TyLE
Neighbor: Sybyr
soundcloud_comment
{'weight': 32}
Profile: Raven's Blood!
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: ISÈAS
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: nimbus.n54
Neighbor: Sybyr
soundcloud_comment
{'weight': 3}
Profile: es
Neighbor: Sybyr
soundcloud_comment
{'weight': 3}
Profile: eternity
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: CursedIsDead
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: mf ed
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: cuff
Neighbor: Sybyr
soundcloud_comment
{'weight': 11}
Profile: killxen 2
Neighbor: Sybyr
soundcloud_comment
{'weight': 20}
Profile: 4ver
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Pro

Profile: yWLq0d
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: NØTHIN MATTERS
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: lil mewtwo
Neighbor: Sybyr
soundcloud_comment
{'weight': 5}
Profile: bohrewton25
Neighbor: Sybyr
soundcloud_comment
{'weight': 4}
Profile: AnthonyAnarchy
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: TRVPLORD SPENCER
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: CRXSSBONE$
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Kaiser Akiyama
Neighbor: Sybyr
soundcloud_comment
{'weight': 3}
Profile: Prod. Kuu
Neighbor: Sybyr
soundcloud_comment
{'weight': 16}
Profile: two29nine
Neighbor: Sybyr
soundcloud_comment
{'weight': 3}
Profile: ٰ
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: dark
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: REALSCOTTYPOPE
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: Marwin
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Sean
Neighbor: Sybyr
sound

Profile: Vanqui$h
Neighbor: Sybyr
soundcloud_comment
{'weight': 3}
Profile: awe mox¡e
Neighbor: Sybyr
soundcloud_comment
{'weight': 4}
Profile: Slowed Down Music
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: ‌
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: IMPUL$E11111
Neighbor: Sybyr
soundcloud_comment
{'weight': 6}
Profile: kadi
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: michaelangel
Neighbor: Sybyr
soundcloud_comment
{'weight': 6}
Profile: MCH6
Neighbor: Sybyr
soundcloud_comment
{'weight': 4}
Profile: VODNIK VOORHEES
Neighbor: Sybyr
soundcloud_comment
{'weight': 3}
Profile: bootlvg
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: PASTELENDO [archives]
Neighbor: Sybyr
soundcloud_comment
{'weight': 8}
Profile: DRILLA THA HERETIC
Neighbor: Sybyr
soundcloud_comment
{'weight': 3}
Profile: Alex B
Neighbor: Sybyr
soundcloud_comment
{'weight': 3}
Profile: zion brae
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: ✨Mister Buddha✨
Neighb

Profile: secretapecharlie
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: fxckupmark
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: dolomub
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Hadiyo Geedi
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: kevinvoorhees
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: BluKol⛧
Neighbor: Sybyr
soundcloud_comment
{'weight': 7}
Profile: GABEYBABEY
Neighbor: Sybyr
soundcloud_comment
{'weight': 7}
Profile: Gxthrips
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Lyrikilla999
Neighbor: Sybyr
soundcloud_comment
{'weight': 12}
Profile: ☫𝖍𝖎𝖊𝖗𝖔𝖕𝖍𝖆𝖓𝖙𝖔𝖒.𝖊𝖝𝖊☫
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: neighborhood arion
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: YNA
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: ghxts
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Roachklip
Neighbor: Sybyr
soundcloud_comment
{'weight': 5}
Profile: Androo Joseff
Ne

Profile: sylveon
Neighbor: Sybyr
soundcloud_comment
{'weight': 12}
Profile: beezlebub
Neighbor: Sybyr
soundcloud_comment
{'weight': 3}
Profile: 0428
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: 3200thrax
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Максим
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Cristiann Val
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: ze - phyr
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: 056kiruto
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: FKA Cherry
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Eddy No$tiliga
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: cathisper
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: Влад Ерошенко
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: ラスターrasta sensei先生
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Jonas Idselis
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: jaqshawty
Neighbor: S

...

Profile: SOREI
Neighbor: Sybyr
soundcloud_comment
{'weight': 8}
Profile: vrotagonist <3
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: checkerx
Neighbor: Sybyr
soundcloud_comment
{'weight': 3}
Profile: sleepfailure
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: maxokoolin
Neighbor: Sybyr
soundcloud_comment
{'weight': 3}
Profile: GorChong
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: ✰BANGEL (FKA XYWULF:SSPOOKED)✰
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: FLXSH2DIE4
Neighbor: Sybyr
soundcloud_comment
{'weight': 3}
Profile: olin
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: MARCUSTHAGAWD
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: null0xd
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: ueouh
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: 51st Citizen
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Haybiad
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Trequan Smith 2
Neig

Profile: Big Roshi
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: YaBoyTorres-No2
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Sleep Kitchen
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Bixenter
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: dummistick
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: PROD.KILLJXRGE
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: 🍕
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Maci zo
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: A5HLEY
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: noah
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Cozy Yogi
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: prod. Cold14
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: Stilts
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: 1 3 7 (@1thirty7)
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: qiuwes
Neighbor: Sybyr
soundcloud_

Profile: NathanielRay
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: HollyBlue
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: David Gibson Jr.
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: DyRu😈
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: 𝔒𝔛𝔜ℭ𝔒ℜ𝔈
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Canoe Crew Kyle
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: tr1st4n
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: ☪︎ RODEOGLO | 000AIKIDO
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: STM
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: $aebaH (sey-buh)
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: viohma
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: fentanyl
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: kittydog
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: Cris Strife
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Chris Gomez
Neighbo

Profile: *ctrl*
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: Lil Addy
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: X U N T I T L E D
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: joe
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: Uporotaya.Transuha
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: User 840895139
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Chez
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: plz rob me
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: SKYE
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: based bwoy
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: Shiki XO
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: MadokaXoxox
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: R O J I  悲しい ®
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎ ‎
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: WVLTUH
Ne

Profile: +prod. levi
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: AurYbae
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: parsa x
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: znakzz
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: stevendied
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: N.Maddie
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: thxqq.thxnos
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Yaguru
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: 3ject_firedrake 2
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: areizaga
Neighbor: Sybyr
soundcloud_comment
{'weight': 2}
Profile: fuck off
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: vain
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: 伊斯梅尔🦁😁
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: JVKXB/hinokami
Neighbor: Sybyr
soundcloud_comment
{'weight': 1}
Profile: Miguel Ventura
Neighbor: Sybyr
soundcloud

# Neo4j using Py2neo

In [21]:
from py2neo import Node, Relationship, Graph
from requests.exceptions import ConnectionError, HTTPError

### Adding functions

In [22]:
# def addNode(profile):
#     query = ('MERGE (profile:soundcloud {id: {profile}.id}) ON CREATE SET profile={profile}')
#     try:
#         userGraph.cypher.execute(query, {'profile': getUserInfo(profile)})
#     except SocketError:
#         print("\t\t\t", "----Cannot connect to cypher db. Assume the query was executed successfully.----")
#     return True

In [24]:
neo4j_graph = Graph() # requires setup of graph database and password before instantiating graph object

ConnectionRefusedError: [Errno 61] Connection refused